In [120]:
#jupyter notebook --browser chrome
#$ pwd
# /Users/jinlin_song/Desktop/Github/h1b_statistics/insight_testsuite/tests/test_1/input

import sys

# Get variables from sys
# data_file = sys.argv[1]
# occupation_file = sys.argv[2]
# state_file = sys.argv[3]

# Read data from input data path
data_file = './insight_testsuite/tests/test_1/input/h1b_input.csv'

In [121]:
class H1B_CSV_dataset:
    def __init__(self, data_file, sep = ';'):
        self.value = True
        self.data_input = []
        try:
            with open(data_file,'r',encoding="utf-8") as f:
                for row in f:
                    self.data_input.append(row.strip('\n').split(sep))
        except:
            print('Error: Not able to load data from given input file.')
            self.value = False
            ### sys.exit()
            
    ### csv validation method
    def is_valid(self):
        return self.value
    
    ### get H1B CSV dataset header
    def data_header(self):
        return self.data_input[0] if self.value else None
    
    ### get H1B CSV dataset without header
    def data_set(self):
        return self.data_input[1:] if self.value else None

In [122]:
H1B_CSV_dataset = H1B_CSV_dataset(data_file,sep = ';')

In [123]:
H1B_CSV_dataset.data_input
H1B_CSV_dataset.is_valid()
data_header = H1B_CSV_dataset.data_header()
data_set = H1B_CSV_dataset.data_set()

In [125]:
def extract_related_columns(data_set, data_header, state_column_name = 'WORKSITE_STATE',
                            occupation_column_name = 'SOC_NAME',
                            status_column_name = 'CASE_STATUS'):
    
    related_column_index = []
    try:
        state_index = data_header.index(state_column_name)
        related_column_index.append(state_index)
    except:
        print('Error: Not able to find state_column_name, please specify correct state_column_name.')
        return False
    try:
        occupation_index = data_header.index(occupation_column_name)
        related_column_index.append(occupation_index)
    except:
        print('Error: Not able to find occupation_column_name, please specify correct occupation_column_name.')
        return False
    try:
        status_index = data_header.index(status_column_name)
        related_column_index.append(status_index)
    except:
        print('Error: Not able to find status_column_name, please specify correct status_column_name.')
        return False
    
    ### extract related columns
    H1B_dataset = [[e[i].strip('"') for i in related_column_index] for e in data_set]
    
    ### remove non-certified records
    H1B_dataset_certified = [e[:2] for e in H1B_dataset if e[2] == 'CERTIFIED']
    
    return H1B_dataset_certified

In [126]:
H1B_dataset_certified = extract_related_columns(data_set, data_header, state_column_name = 'WORKSITE_STATE',
                            occupation_column_name = 'SOC_NAME',
                            status_column_name = 'CASE_STATUS')

In [135]:
H1B_dataset_certified

[['WA', 'SOFTWARE DEVELOPERS, APPLICATIONS'],
 ['CA', 'ACCOUNTANTS AND AUDITORS'],
 ['TX', 'DATABASE ADMINISTRATORS'],
 ['DE', 'SOFTWARE DEVELOPERS, APPLICATIONS'],
 ['AL', 'SOFTWARE DEVELOPERS, APPLICATIONS'],
 ['FL', 'SOFTWARE DEVELOPERS, APPLICATIONS'],
 ['FL', 'SOFTWARE DEVELOPERS, APPLICATIONS'],
 ['MD', 'COMPUTER SYSTEMS ANALYST'],
 ['NJ', 'COMPUTER OCCUPATIONS, ALL OTHER'],
 ['GA', 'SOFTWARE DEVELOPERS, APPLICATIONS']]

In [128]:
class H1B_stat:
    def __init__(self, H1B_dataset_certified):
        self.H1B_dataset_certified = H1B_dataset_certified
        self.total_records = len(H1B_dataset_certified)

    def occupation_stat(self):
        occupation_stat_dict = {}
        for record in self.H1B_dataset_certified:
            if record[1] in occupation_stat_dict:
                occupation_stat_dict[record[1]] += 1
            else:
                occupation_stat_dict[record[1]] = 1
        
        ### sort occupation_stat_dict
        occupation_stat_set = sorted(occupation_stat_dict.items(), key = lambda kv: kv[1], reverse = True)[:10]

        ### add percentaga    
        occupation_stat_set = [[e[0], e[1], float(round(100*e[1]/self.total_records, 1))] for e in occupation_stat_set]    

        return occupation_stat_set
    
    def state_stat(self):
        state_stat_dict = {}
        for record in self.H1B_dataset_certified:
            if record[0] in state_stat_dict:
                state_stat_dict[record[0]] += 1
            else:
                state_stat_dict[record[0]] = 1
                
        ### sort state_stat_dict
        state_stat_set = sorted(state_stat_dict.items(), key = lambda kv: kv[1], reverse = True)[:10]

        ### add percentaga    
        state_stat_set = [[e[0], e[1], float(round(100*e[1]/self.total_records, 1))] for e in state_stat_set]    

        return state_stat_set

In [129]:
H1B_stat = H1B_stat(H1B_dataset_certified)

In [130]:
state_stat = H1B_stat.state_stat()

In [131]:
occupation_stat = H1B_stat.occupation_stat()

In [132]:
occupation_stat

[['SOFTWARE DEVELOPERS, APPLICATIONS', 6, 60.0],
 ['ACCOUNTANTS AND AUDITORS', 1, 10.0],
 ['DATABASE ADMINISTRATORS', 1, 10.0],
 ['COMPUTER SYSTEMS ANALYST', 1, 10.0],
 ['COMPUTER OCCUPATIONS, ALL OTHER', 1, 10.0]]

In [133]:
def write_data(dataset, path, data_type):
    file_header = "NUMBER_CERTIFIED_APPLICATIONS;PERCENTAGE"
    with open(path, 'w') as f:
        if data_type == 'TOP_OCCUPATIONS':
            file_header = "TOP_OCCUPATIONS;" + file_header
        elif data_type == 'TOP_STATES':
            file_header = "TOP_STATES;" + file_header
        else:
            print("Error: Invalid data_type, please specify data_type as 'TOP_OCCUPATIONS' or 'TOP_STATES'.")
            return False
        
        ### write header to the output file
        f.write(file_header + '\n')
        
        ### convert integer to string
        dataset_output = [[e[0], str(e[1]), str(e[2]) + '%'] for e in dataset]
        
        ### write data to the output file
        for item in dataset_output:
            f.write(";".join(item) + '\n')
    return True

In [134]:
path_occupation_stat = "/Users/jinlin_song/Desktop/Github/h1b_statistics/top_10_occupations.txt"
path_state_stat = "/Users/jinlin_song/Desktop/Github/h1b_statistics/top_10_states.txt"

write_data(state_stat, path_state_stat, data_type = 'TOP_STATES')
write_data(occupation_stat, path_occupation_stat, data_type = 'TOP_OCCUPATIONS')

True